### Student Information
Name: 黃俊瑋 

Student ID: 108062308

GitHub ID: [tangerine1202](https://github.com/tangerine1202/DM2022-Lab2-Homework)

Kaggle name: [Chun Wei Huang](https://www.kaggle.com/tangerine1202)

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter by [this link](https://www.kaggle.com/t/2b0d14a829f340bc88d2660dc602d4bd). The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   

    Submit your last submission __BEFORE the deadline (Nov. 22th 11:59 pm, Tuesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the img folder of this repository and rerun the cell Student Information.

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 25th 11:59 pm, Friday)__.

## First Part: Take Home Exercises

- repo link: https://github.com/tangerine1202/DM2022-Lab2-Master

# Second Part: Kaggle Competition

In [2]:
from datetime import datetime 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

from tqdm.notebook import tqdm
# tqdm.pandas()

## Prepare Data

In [ ]:
ident = pd.read_csv('data/data_identification.csv', index_col='tweet_id')
labels = pd.read_csv('data/emotion.csv', index_col='tweet_id')

print('train size', ident[ident['identification'] == 'train'].shape)
print('test  size', ident[ident['identification'] == 'test'].shape)

print(ident.head(3))
print(labels.head(3))

In [ ]:
train_idxs = ident[ident['identification'] == 'train'].index
train_labels = labels.loc[train_idxs, 'emotion']

train_labels.head(3)

In [ ]:
print('labels distribution')
print(train_labels.value_counts())
sns.barplot(x=train_labels.value_counts().index, y=train_labels.value_counts().values)

In [ ]:
data = pd.read_json('data/tweets_DM.json', lines=True)

# drop useless columns (only 1 unique value)
data = data.drop(columns=['_index', '_type'])
source = data['_source']

# parsing tweet
def parse_source(data):
    data = data['tweet']
    parsed_data = {}
    for key, value in data.items():
      parsed_data[key] = value
    return parsed_data
tweet = pd.DataFrame(source.map(parse_source).tolist())
data = pd.concat([data, tweet], axis=1)
data = data.drop(columns=['_source'])

data.set_index('tweet_id', inplace=True)

data.head(3)

In [ ]:
train_data = data.loc[train_idxs, :].sort_values(by='tweet_id')
test_data = data.loc[~data.index.isin(train_idxs), :].sort_values(by='tweet_id')
train_labels = train_labels.sort_index()

assert len(train_data) == len(train_labels)
assert np.all(train_data.index == train_labels.index)
print(len(data))
print(len(train_data), len(test_data))

In [ ]:
train_data.to_pickle('data/train_data.pkl')
train_labels.to_pickle('data/train_labels.pkl')
test_data.to_pickle('data/test_data.pkl')

In [ ]:
# FIXME: lost some rows in training data when reading from tsv
# train_data.to_csv('data/train_data.tsv', sep='\t', encoding='utf-8')
# train_labels.to_csv('data/train_labels.tsv', sep='\t', encoding='utf-8')
# test_data.to_csv('data/test_data.tsv', sep='\t', encoding='utf-8')

## Exploratory Data Analysis

### What `_score` means?

In [ ]:
print(data['_score'].describe())
sns.histplot(data['_score'])

### Hashtag

## Transfer Learning from Hugging Face 

In [3]:
from torch.utils.data import DataLoader
from torch.optim import AdamW

from datasets import Dataset, load_dataset
from datasets import load_metric
import evaluate

from transformers import AutoTokenizer
from transformers import AutoModel, AutoConfig, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import get_scheduler
from transformers import DataCollatorWithPadding
from transformers.modeling_outputs import TokenClassifierOutput

from transformers import XLNetTokenizer, XLNetForSequenceClassification

2022-11-15 01:53:11.208198: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 01:53:11.351321: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-15 01:53:11.839147: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alan/.cuda/lib64:/home/alan/.cuda/extras/CUPTI/lib64:/home/alan/.cuda/lib64:/home/alan/.cuda/extras/CUPTI/lib64:
2022-11-15 01:53:11.839304: W tensorflow/stream_executor/platform/default/dso_loader.

In [ ]:
# data
TRAIN_DATA_PATH = 'data/train_data.pkl'
TRAIN_LABEL_PATH = 'data/train_labels.pkl'
TEST_DATA_PATH = 'data/test_data.pkl'
EMOTION_NAMES = ['joy', 'anticipation', 'trust' , 'sadness' , 'disgust' , 'fear' , 'surprise', 'anger']

# constants
MODEL_NAME = 'bert-base-cased'
NUM_LABELS = 8

# hyper-parameters
EPOCHS = 3
BATCH_SIZE = 8

In [ ]:
emotion2id = {emotion: i for i, emotion in enumerate(EMOTION_NAMES)}
id2emotion = {i: emotion for i, emotion in enumerate(EMOTION_NAMES)}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
df_train_X = pd.read_pickle(TRAIN_DATA_PATH)
df_train_y = pd.read_pickle(TRAIN_LABEL_PATH)
assert len(df_train_X) == len(df_train_y)
df_train = pd.concat([df_train_X, df_train_y], axis=1)

# numerical labels
df_train['label'] = df_train['emotion'].map(lambda x: emotion2id[x])

df_train.sample(5)

In [ ]:
dataset = Dataset.from_pandas(df_train[['text', 'label']])

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.save_to_disk(f'data/{MODEL_NAME}_tokenized_datasets')

In [ ]:
tokenized_datasets = Dataset.load_from_disk(f'data/{MODEL_NAME}_tokenized_datasets')

In [ ]:
small_train_size = 50000
small_eval_size = 10000
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(small_train_size))
small_eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(small_train_size, small_train_size + small_eval_size))

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)

In [ ]:
training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
)

In [ ]:
f1_metric = evaluate.load("f1")
acu_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    f1_macro = f1_metric.compute(predictions=predictions, references=labels, average='macro')['f1']
    acu = acu_metric.compute(predictions=predictions, references=labels)['accuracy']
    return {'f1_macro': f1_macro, 'acu': acu}


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained(f'model/{MODEL_NAME}-50000')

## Evaluation

In [ ]:
# classifier = pipeline(
#     'sentiment-analysis', 
#     model=model, 
#     tokenizer=tokenizer,
#     device=0,
# )

In [ ]:
# tdf = pd.DataFrame(classifier(small_eval_dataset['text']))
# tdf['answer'] = pd.DataFrame(small_eval_dataset['label'])
# tdf['pred'] = tdf['label']
# tdf.drop(columns=['label'], inplace=True)

# tdf

In [ ]:
# tdf['results'] = tdf['pred'].map(lambda s: int(s[-1])) == tdf['answer']
# tdf['results'] = tdf['results'].astype(float)
# tdf['results'].describe()

## Inference

In [ ]:
TEST_BATCH_SIZE = 64

In [ ]:
df_test = pd.read_pickle(TEST_DATA_PATH)
df_test.sample(5)

In [ ]:
classifier = pipeline(
    'sentiment-analysis', 
    model=model, 
    tokenizer=tokenizer,
    device=0,
)

In [ ]:
end_idx = None

# prediction
df_result = pd.DataFrame(classifier(df_test['text'][:end_idx].tolist(), batch_size=TEST_BATCH_SIZE))
# tweet_id as id
df_result['id'] = df_test.index[:end_idx]
# convert label to emotion
df_result['emotion'] = df_result['label'].map(lambda s: id2emotion[int(s[-1])])

df_result.head(3)

In [ ]:
df_result[['tweet_id', 'emotion']].to_csv(f'submission/01.csv', index=False)

# Custom model

In [4]:
# data
TRAIN_DATA_PATH = 'data/train_data.pkl'
TRAIN_LABEL_PATH = 'data/train_labels.pkl'
TEST_DATA_PATH = 'data/test_data.pkl'
EMOTION_NAMES = ['joy', 'anticipation', 'trust' , 'sadness' , 'disgust' , 'fear' , 'surprise', 'anger']

# constants
MODEL_NAME = 'cardiffnlp/twitter-roberta-base-emotion'
NUM_LABELS = 8

# hyper-parameters
EPOCHS = 5
BATCH_SIZE = 8
LR = 5e-5

In [5]:
emotion2id = {emotion: i for i, emotion in enumerate(EMOTION_NAMES)}
id2emotion = {i: emotion for i, emotion in enumerate(EMOTION_NAMES)}

In [6]:
df_train_X = pd.read_pickle(TRAIN_DATA_PATH)
df_train_y = pd.read_pickle(TRAIN_LABEL_PATH)
assert len(df_train_X) == len(df_train_y)
df_train = pd.concat([df_train_X, df_train_y], axis=1)

# numerical labels
df_train['label'] = df_train['emotion'].map(lambda x: emotion2id[x])

df_train.sample(5)

,_score,_crawldate,hashtags,text,emotion,label
tweet_id,,,,,,
0x1e8497,362,2017-04-26 18:09:52,"[Men, life, addiction, desire, attraction, ten...",#Men are too important in my #life. But what's...,joy,0
0x2004b0,1020,2016-01-05 09:19:21,[inspiration],Believe you can and you are halfway there. - T...,anticipation,1
0x28e8a0,1,2015-05-02 22:50:02,[buttrue],I feel like with the current state of the affa...,sadness,3
0x2f8cc0,548,2017-02-24 18:53:49,[],"Luke 13-5 I tell you, no; but unless you repen...",anticipation,1
0x31833a,123,2017-04-05 20:43:04,"[property, management, condo, assessement]",@ellsavvy Has anyone heard of a #property #man...,joy,0


In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
# dataset = Dataset.from_pandas(df_train[['text', 'label']])

In [9]:
# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# tokenized_datasets.save_to_disk(f'data/{MODEL_NAME}_tokenized_datasets')

In [10]:
tokenized_datasets = Dataset.load_from_disk(f'data/{MODEL_NAME}_tokenized_datasets')

In [11]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.remove_columns(["text", 'tweet_id'])
tokenized_datasets.set_format("torch")
tokenized_datasets

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1455563
})

In [12]:
small_train_size = 50#0000
small_eval_size = 10#000
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(small_train_size))
small_eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(small_train_size, small_train_size + small_eval_size))

Loading cached shuffled indices for dataset at data/cardiffnlp/twitter-roberta-base-emotion_tokenized_datasets/cache-874c557c48086f6c.arrow
Loading cached shuffled indices for dataset at data/cardiffnlp/twitter-roberta-base-emotion_tokenized_datasets/cache-874c557c48086f6c.arrow


In [13]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=BATCH_SIZE, collate_fn=data_collator)

## Prepare model

In [14]:
# ref: https://towardsdatascience.com/adding-custom-layers-on-top-of-a-hugging-face-model-f1ccdfc257bd
class CustomModel(torch.nn.Module):
  def __init__(self, checkpoint, num_labels): 
    super(CustomModel,self).__init__() 
    self.num_labels = num_labels 

    #Load Model with given checkpoint and extract its body
    self.model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    self.dropout = torch.nn.Dropout(0.1) 
    self.classifier = torch.nn.Linear(768, num_labels) # load and initialize weights

  def forward(self, input_ids=None, attention_mask=None, labels=None):
    #Extract outputs from the body
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

    #Add custom layers
    sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state

    logits = self.classifier(sequence_output[:,0,:].view(-1,768)) # calculate losses
    
    loss = None
    if labels is not None:
      loss_fct = torch.nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    
    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [15]:
model = CustomModel(MODEL_NAME, NUM_LABELS).to(device)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-emotion were not used when initializing RobertaModel: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
optimizer = AdamW(model.parameters(), lr=LR)

num_training_steps = EPOCHS * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
f1_metric = evaluate.load("f1")
acu_metric = evaluate.load("accuracy")

def compute_metrics():
    f1_macro = f1_metric.compute(average='macro')['f1']
    acu = acu_metric.compute()['accuracy']
    return {'f1_macro': f1_macro, 'acu': acu}


## Training

In [ ]:
progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(EPOCHS * len(eval_dataloader)))


for epoch in range(EPOCHS):
  model.train()
  for batch in train_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar_train.update(1)

  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    f1_metric.add_batch(predictions=predictions, references=batch["labels"])
    acu_metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar_eval.update(1)
    
  print(compute_metrics())

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'f1_macro': 0.16666666666666666, 'acu': 0.3}
{'f1_macro': 0.16666666666666666, 'acu': 0.3}
{'f1_macro': 0.16666666666666666, 'acu': 0.3}
{'f1_macro': 0.16666666666666666, 'acu': 0.3}
{'f1_macro': 0.16666666666666666, 'acu': 0.3}


## Evaluation

In [19]:
f1_metric = evaluate.load("f1")
acu_metric = evaluate.load("accuracy")

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    f1_metric.add_batch(predictions=predictions, references=batch["labels"])
    acu_metric.add_batch(predictions=predictions, references=batch["labels"])

print(compute_metrics())

{'f1_macro': 0.275, 'acu': 0.4}


## Inference

In [20]:
TEST_BATCH_SIZE = 64

In [21]:
df_test = pd.read_pickle(TEST_DATA_PATH)
df_test.sample(5)

,_score,_crawldate,hashtags,text
tweet_id,,,,
0x372160,568,2016-05-21 22:06:38,"[baby, parenting, maternity, mumlife, Motherhood]",Got the tiredest #baby ever over here. Waking ...
0x365823,859,2016-05-25 18:00:57,[],"The most high is always by myside,giving thnks..."
0x23ea61,179,2016-04-03 16:50:10,[],Okay okay I get that this is controversial and...
0x2c5fc4,473,2015-08-10 07:25:31,[],@mostlygeordie @StAustellBrew Prefer a proper ...
0x31f375,929,2017-02-09 21:07:25,[],@paatriciaa43 But did you actually meet him th...


In [22]:
test_dataset = Dataset.from_pandas(df_test)
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=['tweet_id', '_score', '_crawldate', 'hashtags'])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask"])
data_collator = DataCollatorWithPadding(tokenizer)


Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
































































































100%|██████████| 35/35 [00:19<00:00, 16.93it/s]













100%|█████████▉| 411/412 [00:14<00:00, 28.85ba/s]


In [23]:
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, collate_fn=data_collator)

In [25]:
df_result = pd.DataFrame({'id': df_test.index})
df_result['label'] = np.zeros(len(df_result)) - 1

idx = 0
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    inc = min(TEST_BATCH_SIZE, len(df_result) - idx)
    df_result.loc[idx:idx+inc-1, 'label'] = predictions.cpu().numpy()
    idx += inc

# convert label to emotion
df_result['emotion'] = df_result['label'].map(lambda x: id2emotion[x])

df_result.head(3)

KeyboardInterrupt: 

In [ ]:
df_result[['id', 'emotion']].to_csv('submission/03.csv', index=False)